In [ ]:
%reset -f

In [ ]:
def is_main_module():
    """
    Returns whether this notebook is the main module
    i.e. not being run from anotherr notebook
    """
    return __name__ == '__main__' and '__file__' not in globals()

SAVE_FIGURES = False
SAVE_FIGURE_DPI = 300
SAVE_FIGURE_FORMAT = "PDF"  # PDF, SVG, PNG, for saved images

In [ ]:
%matplotlib inline
%config Inlinebackend.figure_format = 'svg'

In [ ]:
#import libraries
import sys
# add the path to the Python module directory
sys.path.insert(0, '../Python')

import matplotlib.pyplot as plt
import numpy as np
import math

import Constants
import Utils

import SphericalEarthModel as SEM
import Prf as PRF
import Nesz as NESZ
import Aasr as AASR
import Rasr as RASR

import planararrayantenna as PAA

### System Parameters

In [ ]:
# system parameters
Frequency       = 9.65e9
Wavelength      = Constants.SPEED_OF_LIGHT / Frequency
Altitude        = 514e3
TxPowerPeak     = 2260
TxDutyCycle     = 0.18
Prf             = 3210
AntennaOffset   = 33.8      # Antenna Mounting angle/Mid-Look angle in degrees
NoiseFigure     = 4.3       # TRM Noise figure in dB
Losses          = 4.1       # System losses: Processing atmosphere, taper, degradation, etc. in dB
ProcessedAzimuthBandwidth = 2266  # Doppler bandwidth in Hz

# antenna parameters
antLength       = 4.8
antHeight       = 0.8
elNumElements   = 32
azNumElements   = 12
efficiency      = 0.7 #0.886
    
# System requirements
ReqNesz         = -20       # Required NESZ in dB
ReqRasr         = -20       # Required RASR in dB
ReqAasr         = -20       # Required AASR in dB
ReqResolutionAz = 3         # Required Azimuth resolution in meters   (cross-range) 
ReqResolutionRg = 3         # Required Elevation resolution in meters (ground range)
ReqGroundSwath  = 30e3      # Required Ground swath width in meters

# Derived parameters
MidSwathIncidenceAngle = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))
ChirpBandwidth         = Constants.SPEED_OF_LIGHT / (2*ReqResolutionRg*np.sin(MidSwathIncidenceAngle))    
TxPowerAvg             = TxPowerPeak * TxDutyCycle
PulseLen               = TxDutyCycle / Prf
CompressionGain        = (PulseLen * ChirpBandwidth)

#------------------------------------------------------------
# geometry code
IncidenceAngleRange = np.arange(15, 50.1, 0.1) # Scene incidence angle range in degrees
IncidenceAngleRangeRad = IncidenceAngleRange * (np.pi/180)

LookAngleRangeRad = SEM.LookAngle(Altitude, IncidenceAngleRangeRad)
LookAngleRange = LookAngleRangeRad * (180/np.pi)

MidSwathLookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)

#### Ground swath vs Elevation Beamwidth

In [ ]:
if is_main_module():
    # Get the ground swath width for the different look angles and TX elevation beamwidths
    Test_el_bw_range = np.arange(1, 61, 1)          # beamwidth ranges between 1 and 60 degrees
    Test_look_angles = np.arange(20, 40, 5)         # elevation look angle ranges between 20 and 35 degrees
    Test_look_angles = np.append(Test_look_angles, 33.8)
    Test_altitudes   = np.asarray([Altitude])          # test altitudes. Min, mid and Max altitudes

    Test_altitude_swaths = []
    for alt in Test_altitudes:
        # for every altitude calculate the different swath over the look angle ranges, for the different beam widths
        Test_ground_swaths = [] 
        for la in Test_look_angles:
            _, _ground_swath = SEM.SwathGroundSwathFromBeamwidth(alt, Utils.deg2rad(la), Utils.deg2rad(Test_el_bw_range))
            Test_ground_swaths.append(_ground_swath)
        Test_altitude_swaths.append(Test_ground_swaths)
    Test_altitude_swaths = np.asarray(Test_altitude_swaths)

    # Plot the ground swath widths vs beamwidths & look angles
    fig, ax1 = plt.subplots(1,1,figsize = (12,6))
    for i in range(Test_altitude_swaths.shape[0]):
        for j in range(Test_altitude_swaths[i].shape[0]):
            ax1.plot(Test_el_bw_range, Test_altitude_swaths[i][j,:] / 1000, label=r"Look Angle = %.2f deg" % (Test_look_angles[j]))
            
    ax1.legend(loc=2)
    ax1.set_xlabel(r"Elevation Beamwidth (deg)")
    ax1.set_ylabel(r"Ground swath Width (km)")
    ax1.grid(linestyle='dotted')
    ax1.set_ylim(0, 100)
    ax1.set_xlim(1,6)
    ax1.axvline(x=2.18, linestyle="dotted")
    ax1.axhline(y=ReqGroundSwath/1000, linestyle="dotted")
    plt.tight_layout()
    
    ReqTxBeamwidth_nom = 2.18
    sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_altitudes, 
                                            Utils.deg2rad(AntennaOffset), 
                                            Utils.deg2rad(ReqTxBeamwidth_nom))
    print("Swath = {:.2f} km and Ground swath = {:.2f} km @ Look angle = {} deg and Elevation BW = {} deg".format(sw[0]/1000, gw[0]/1000, AntennaOffset, ReqTxBeamwidth_nom))

#### Ground swath and incidence angle estimation

In [ ]:
# ground swath estimation
laMin = AntennaOffset - (ReqTxBeamwidth_nom / 2)
laMax = AntennaOffset + (ReqTxBeamwidth_nom / 2)
    
lowLa = np.deg2rad(laMin)
highLa = np.deg2rad(laMax)

lowIa = SEM.IncidenceAngle(Altitude, lowLa)
highIa = SEM.IncidenceAngle(Altitude, highLa)

lowSR = SEM.SlantRange(Altitude, lowIa, lowLa)
highSR = SEM.SlantRange(Altitude, highIa, highLa)
print("Swath: {:.3f} km".format((highSR- lowSR)/1000))

lowGR = SEM.IncidenceAngleToGroundRange(Altitude, lowIa)
highGR = SEM.IncidenceAngleToGroundRange(Altitude, highIa)
swath = highGR - lowGR
print("Ground Swath: {:.3f} km".format(swath/1000))

### PRF Analysis

#### PRF vs Altitude

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    altitudes = np.arange(450e3, 555e3, 5e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, altitudes)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(altitudes)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(altitudes, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             altitudes, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadir limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(altitudes/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(altitudes/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(altitudes/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(altitudes/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(altitudes/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(altitudes/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(altitudes/1000, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(altitudes/1000, prfMax, color='tab:orange', linestyle='dashed') 
    # plot the altitude and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=Altitude/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Altitude')
    ax1.set_xlabel(r'Altitude (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(3000, 4500)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)

    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05a_aesa_verification_01_prf_vs_alt.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05a_aesa_verification_01_prf_vs_alt.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05a_aesa_verification_01_prf_vs_alt.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Antenna Length

In [ ]:
# define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses   = 32
    numNadirEchoes = 12
    
    # define the antenna length range to cover
    antennaLength = np.arange(2, 6.1, 0.1)
    
    # get the minimum PRF
    prfMin = []
    for ant in (antennaLength):
        prfMin.append(PRF.PrfMin(ant, Altitude))
    prfMin = np.asarray(prfMin)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay:            {:.6f} s".format(nadirDelay))
    print("Near range Delay (tn):  {:.6f} s".format(nearDelay))
    print("Far range Delay (tf):   {:.6f} s".format(farDelay))
    print("tf - tn:                {:.6f} s".format(farDelay - nearDelay))
    print("PRF max:                {:.6f} Hz".format(prfMax))  
    print("PRF min ({:.2}m antenna): {:.6f} Hz".format(antLength, PRF.PrfMin(antLength, Altitude)))  
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.axhline(y=nadirLower, color="red", alpha=0.6)
        ax1.axhline(y=nadirUpper, color="red", alpha=0.6)
        plt.fill_between(antennaLength, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.axhline(y=prfLower, color="black", alpha=0.6)
        ax1.axhline(y=prfUpper, color="black", alpha=0.6)
        plt.fill_between(antennaLength, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(antennaLength, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.axhline(y=prfMax, color='tab:orange', linestyle='dashed') 
    # plot antenna length and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle='dashed') 
    ax1.axvline(x=antLength, color='black', linestyle='dashed') 
    
    
    #ax1.set_title('PRF vs Antenna length')
    ax1.set_xlabel(r'Antenna length (m)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(3000, 4500)
    plt.grid(linestyle='dotted')
    ax1.margins(0)
    plt.tight_layout()
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05b_aesa_verification_01_prf_vs_ant.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05b_aesa_verification_01_prf_vs_ant.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05b_aesa_verification_01_prf_vs_ant.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Pulse Length

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the pulse length range to cover
    pulseLengths = np.arange(0, 101e-6, 1e-6)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             pulseLengths)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       pulseLengths,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(pulseLengths, (farDelay - nearDelay))
    
    # display/plot the results
    print("Near range delay: {:.6f} s".format(nearDelay))
    print("Far range delay:  {:.6f} s".format(farDelay))
    print("Nadir delay:      {:.6f} s".format(nadirDelay))
    print("Minimum PRF:      {:.2f} Hz".format(prfMin))
    
    # plot PRF vs pulse length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(pulseLengths* 1e6, nadirLower, color="red", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(pulseLengths* 1e6, prfLower, color="black", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, prfUpper, color="black", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(pulseLengths* 1e6, prfMax, color='tab:orange', linestyle='dashed')  
    # plot pulse length and ideal PRF
    ax1.axvline(x=PulseLen*1e6, color='black', linestyle='dashed')  
    ax1.axhline(y=Prf, color='black', linestyle='dashed')  
    
    #ax1.set_title('PRF vs Pulse Lengh')
    ax1.set_xlabel(r'Pulse Lengh, $\tau_p$ ($\mu$s)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(3000, 4500)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)

    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05c_aesa_verification_01_prf_vs_pul.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05c_aesa_verification_01_prf_vs_pul.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05c_aesa_verification_01_prf_vs_pul.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Swath Width

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    groundSwaths = np.arange(5e3, 51e3, 1e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = Utils.deg2rad(AntennaOffset)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(groundSwaths,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(groundSwaths/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(groundSwaths/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(groundSwaths/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(groundSwaths/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(groundSwaths/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(groundSwaths/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(groundSwaths/1000, prfMax, color='tab:orange', linestyle='dashed')     
    
    # plot ideal ground swath and prf
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=ReqGroundSwath/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Ground Swath')
    ax1.set_xlabel(r'Ground swath, $W_{gr}$ (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(3000, 4500)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05d_aesa_verification_01_prf_vs_swa.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05d_aesa_verification_01_prf_vs_swa.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05d_aesa_verification_01_prf_vs_swa.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Incidence Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 41
    numNadirEchoes = 25
    
    # define the incidence angle (in degreees) ranges to use 
    incidenceAngles = Utils.deg2rad(np.arange(15, 71, 1))

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, incidenceAngles)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    
    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([20, 30, 40, 50, 60, 60])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        # convert look angle to incidence angle
        IA = SEM.IncidenceAngle(altitude, Utils.deg2rad(x))
        V = SEM.IncidenceAngleToGroundRange(altitude, IA)
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")
    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(incidenceAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(Utils.rad2deg(incidenceAngles), prfMax, color='tab:orange', linestyle='dashed')     
    # plot the incidence angle and ideal PRF
    ax1.axvline(x=Utils.rad2deg(MidSwathIncidenceAngle), color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    ax1.axvline(x=Utils.rad2deg(lowIa), color="blue", linestyle="dashed")
    ax1.axvline(x=Utils.rad2deg(highIa), color="blue", linestyle="dashed")
    
    #ax1.set_title('PRF vs Incidence Angle')
    ax1.set_xlabel(r'Incidence Angle, $\theta_{i,m}$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2500, 4500)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05e_aesa_verification_01_prf_vs_inc.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05e_aesa_verification_01_prf_vs_inc.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05e_aesa_verification_01_prf_vs_inc.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

#### PRF vs Look Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 53
    numNadirEchoes = 36
    
    # define the look angle (in degreees) ranges to use 
    lookAngles = LookAngleRangeRad
    
    # calculate the incidence angles from these look angles
    incidenceAngles = IncidenceAngleRangeRad

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngles)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.2f} us".format(nadirDelay * 1e6))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))   
    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(lookAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(lookAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRF Maximum
    ax1.plot(Utils.rad2deg(lookAngles), prfMax, color='tab:orange', linestyle='dashed')   
    # plot antenna mounting angle and ideal prf
    ax1.axvline(x=AntennaOffset, color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    ax1.axvline(x=laMin, color="blue", linestyle="dashed")
    ax1.axvline(x=laMax, color="blue", linestyle="dashed")
    
    #ax1.set_title('PRF vs Look Angle')
    ax1.set_xlabel(r'Look Angle, $\gamma_m$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(3000, 4500)
#     ax1.set_xlim(13, 46)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_05f_aesa_verification_01_prf_vs_loo.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_05f_aesa_verification_01_prf_vs_loo.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_05f_aesa_verification_01_prf_vs_loo.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### NESZ

In [ ]:
# narrow swath TerraSAR-X NESZ verification
if is_main_module():    
    #------------------------------------------------------------
    # TX Antenna code
    #desiredElvElemSpacing = antHeight / elNumElements
    desiredElvElemSpacing = 0.575 * Wavelength
    print(desiredElvElemSpacing)
    #desiredAzElemSpacing = antLength / azNumElements
    desiredAzElemSpacing = antLength / azNumElements
    
    
    # initialise the TX antenna object
    terraSarTxAnt = PAA.PlanarArrayAntenna(el_elem_spacing = desiredElvElemSpacing, 
                                           num_el_elems    = elNumElements,
                                           num_az_elems    = azNumElements, 
                                           az_elem_spacing = desiredAzElemSpacing,
                                           efficiency      = efficiency)
    print(Utils.rad2deg(terraSarTxAnt.elevation_beamwidth(Wavelength)))
    
    #------------------------------------------------------------
    # RX Antenna code
    # initialise the RX antenna object
    terraSarRxAnt = PAA.PlanarArrayAntenna(num_el_elems    = elNumElements, 
                                           el_elem_spacing = desiredElvElemSpacing, 
                                           num_az_elems    = azNumElements, 
                                           az_elem_spacing = desiredAzElemSpacing,
                                           efficiency      = efficiency)
    
    #------------------------------------------------------------
    # antenna pattern for TX and RX antennas
    AntennaPatternRange = np.arange(-90, 90, 0.1)

    # Azimuth angles used only to visualise the antenna pattern in azimuth
    azimuthAngles = np.arange(-5, 5, 0.01)   # azimuth angles in degrees

    # determine if there are grating lobes in elevation (downwards steering angle)
    LaMin = LookAngleRange.min()
    SaMin = LaMin - AntennaOffset
    LaMax = LookAngleRange.max()
    SaMax = LaMax - AntennaOffset
    SaRange = np.arange(SaMin, SaMax, 0.1)

    # TX antenna power patterns
    txElvPat = terraSarTxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
    txAzPat = terraSarTxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)

    # RX antenna power patterns
    rxElvPat = terraSarRxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
    rxAzPat = terraSarRxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)

    # TX / RX antenna gain scaling factor
    gainScale = terraSarTxAnt.gain(Wavelength) / terraSarRxAnt.gain(Wavelength)
    
    #------------------------------------------------------------
    # Calculate the antenna patterns used for calculating the NESZ
    NumSamples = 100
    NumRxScanAngles = 1
    NumTxScanAngles = 31

    txAnt = terraSarTxAnt
    rxAnt = terraSarRxAnt

    RxAntPatterns, TxAntPatterns, rxtxAnglesRad, rxPeaks, RxAntPatMaxAngles, RxAntPatMaxValues \
        = NESZ.CalNeszAntennaPatterns_alt(rxAnt, txAnt, Wavelength, NumRxScanAngles, NumTxScanAngles, 
                                      LookAngleRangeRad.min(), LookAngleRangeRad.max(), NumSamples)
    
    #------------------------------------------------------------
    # calculate the incidence angles and slant ranges over the antenna beam pattern
    IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(NumRxScanAngles * NumTxScanAngles, Altitude, rxtxAnglesRad, LookAngleRangeRad.min())

    #------------------------------------------------------------
    # calculate the core angles subtended by the slant ranges, and from these the ground ranges
    GR_Arr = NESZ.CalcGroundRange(NumRxScanAngles * NumTxScanAngles, Altitude, SR_Arr)

    #------------------------------------------------------------
    # Calulate the value of NESZ of the look angles
    NESZ_Arr, Nesz_max_incidence_angles, Nesz_max_values \
        = NESZ.CalcNesz(NumRxScanAngles * NumTxScanAngles, 
                         Altitude, 
                         ChirpBandwidth, 
                         IA_Arr, SR_Arr, 
                         TxAntPatterns, RxAntPatterns, 
                         Wavelength, TxPowerPeak, 
                         PulseLen, Prf)

    Nesz_dB_Arr = 10*np.log10(NESZ_Arr)
    
    #------------------------------------------------------------
    # plot the NESZ
    fig, ax1 = plt.subplots(figsize = (8, 4))
    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45., 50., 55, 60])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--", label="Calculated NESZ")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--")
    ax1.plot(Utils.rad2deg(Nesz_max_incidence_angles/1000), 10*np.log10(Nesz_max_values), color="green", label=r"Effective NESZ")

    # plot required NESZ value +- 2dB
    ax1.axhline(ReqNesz, color="blue", label="Required NESZ")
    
    ax1.axvline(x=Utils.rad2deg(lowIa), color="blue", linestyle="dashed")
    ax1.axvline(x=Utils.rad2deg(highIa), color="blue", linestyle="dashed")

    # ax1.set_title("NESZ vs Incidence Angle")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"NESZ (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()

    plt.tight_layout()
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_06_aesa_verification_01_nesz.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_06_aesa_verification_01_nesz.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_06_aesa_verification_01_nesz.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### AASR

In [ ]:
# AASR calculations
if is_main_module():
    NumAmbiguities = 10
    FrequencySteps = 100

    # Get the processed Doppler bandwidth
    DopplerBandwidth = [SEM.ProcessedDopplerBandwidth(SEM.PlatformVelocity(Altitude), ReqResolutionAz), 2266.]
    print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth[0]))
    print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth[1]))

    # DopplerFrequencies = np.linspace(-DopplerBandwidth/2, DopplerBandwidth/2, FrequencySteps)
    DopplerFrequencies = np.linspace(start=-10000, stop=10000, num=1000)

    AzimuthAngles = SEM.AzimuthAngleFromDoppler(DopplerFrequencies, Wavelength, SEM.PlatformVelocity(Altitude))

    # calculate the TX/RX antenna patterns over the Doppler frequency bandwidth
    txAzPat = terraSarTxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
    rxAzPat = terraSarRxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
    
    #------------------------------------------------------------
    # Calculate the AASR values
    Aasr_Arr = [AASR.Get_AASR(DopplerBandwidth = DopplerBandwidth[0],
                         Frequencies = DopplerFrequencies,
                         NumAmbiguities = NumAmbiguities,
                         FreqSteps = FrequencySteps,
                         Prf = Prf,
                         TxAntennaPattern = txAzPat,
                         RxAntennaPattern = rxAzPat,
                         TxGain = terraSarTxAnt.gain(Wavelength),
                         RxGain = terraSarRxAnt.gain(Wavelength) ),
                
                AASR.Get_AASR(DopplerBandwidth = DopplerBandwidth[1],
                         Frequencies = DopplerFrequencies,
                         NumAmbiguities = NumAmbiguities,
                         FreqSteps = FrequencySteps,
                         Prf = Prf,
                         TxAntennaPattern = txAzPat,
                         RxAntennaPattern = rxAzPat,
                         TxGain = terraSarTxAnt.gain(Wavelength),
                         RxGain = terraSarRxAnt.gain(Wavelength) ) ]

    
    # create dummy gincidence angle array over which to plot the AASR data
    SwathIaArr = np.linspace(np.asarray(IA_Arr[0]).min(), np.asarray(IA_Arr[len(IA_Arr)-1]).max(), 50)
    
    # create an array equal to the incidence angle array and fill with the AASR data    
    Aasr_Arr_Full = [np.full((SwathIaArr.shape[0]), 10*np.log10(Aasr_Arr[0])),
                     np.full((SwathIaArr.shape[0]), 10*np.log10(Aasr_Arr[1]))]
    
    #------------------------------------------------------------
    # plot the AASR vs ground swath
    fig, ax1 = plt.subplots(1,1, figsize = (8,4))    
    ax1.plot(np.rad2deg(SwathIaArr), Aasr_Arr_Full[0], color='tab:red', label=r"Calculated AASR with $B_p$ = {:.2f} Hz".format(DopplerBandwidth[0]))  
    ax1.plot(np.rad2deg(SwathIaArr), Aasr_Arr_Full[1], color='tab:orange', label=r"Calculated AASR with $B_p$ = {:.2f} Hz".format(DopplerBandwidth[1]))  
    
    ax1.axhline(y=ReqAasr, label="Required AASR", color="blue")
    
    # ax1.set_title('Azimuth-Ambiguity-to-Signal Ratio vs Slant Range')
    ax1.set_ylabel(r'AASR (dB)')
    ax1.set_xlabel(r'Incidence Angle (deg)')
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.set_ylim(-40, -17)
    ax1.margins(0)
    ax1.legend()
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_08_aesa_verification_01_aasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_08_aesa_verification_01_aasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_08_aesa_verification_01_aasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### RASR

In [ ]:
if is_main_module():
    NumSamples = 100
    NumRxScanAngles = 1
    NumTxScanAngles = 31
    NF = 5
    NN = 3
    prf = Prf

    txAnt = terraSarTxAnt
    rxAnt = terraSarRxAnt

    #------------------------------------------------------------
    # firstly get the antenna patterns and anges for the main antenna beams
    RxAntPatterns, TxAntPatterns, \
    rxtxAnglesRad, rxPeaks, \
    RxAntPatMaxAngles, RxAntPatMaxValues \
        = NESZ.CalNeszAntennaPatterns_alt(
            rxAnt, 
            txAnt, 
            Wavelength, 
            NumRxScanAngles, 
            NumTxScanAngles,
            LookAngleRangeRad.min(), 
            LookAngleRangeRad.max(), 
            NumSamples)
    
    # Plot the main antenna patterns
#     fig, ax1 = plt.subplots(figsize = (8,3))
#     for i in range(NumRxScanAngles * NumTxScanAngles):
#         if i == 0:
#             ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 10*np.log10(TxAntPatterns[i]), color="red", label="TX Elevation")
#             ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed", label="RX Elevation")
#         else:
#             ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 10*np.log10(TxAntPatterns[i]), color="red")
#             ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed")

#     ax1.set_title("3dB Antenna pattern vs Angle")
#     ax1.set_xlabel(r"Angle (deg)")
#     ax1.set_ylabel(r"Antenna pattern (dB)")
#     ax1.grid(linestyle='dotted')
#     plt.tight_layout()
#     ax1.legend()
#     plt.show()

    #------------------------------------------------------------
    # calculate the incidence angles and slant ranges over the antenna beam pattern
    IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
        NumRxScanAngles * NumTxScanAngles, 
        Altitude, 
        rxtxAnglesRad, 
        LookAngleRangeRad.min())

    #------------------------------------------------------------
    # calculate the main RASR values for each slant range
    RasrMain = RASR.Get_RasrMain(
        SlantRange = SR_Arr,
        IncidenceAngle = IA_Arr,
        TxAntennaPattern = np.asarray(TxAntPatterns), 
        RxAntennaPattern = np.asarray(RxAntPatterns) )
    RasrMain_dB = 10*np.log10(RasrMain)
    
    # Plot RASR Main values
#     fig, ax1 = plt.subplots(figsize = (8, 3))
#     for i in range(NumRxScanAngles * NumTxScanAngles):
#         if i == 0:
#             ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--", label="RASR")
#         else:
#             ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--")

#     ax1.axhline(ReqNesz, color="blue", label="Required RASR")
#     ax1.set_title("RASR vs Slant Range")
#     ax1.set_xlabel(r"Incidence Angle (deg)")
#     ax1.set_ylabel(r"RASR (dB)")
#     ax1.grid(linestyle='dotted')
#     ax1.legend()
#     plt.tight_layout()
#     plt.show()

    #------------------------------------------------------------
    # Calculate the slant ranges, incidence angles, and RX/TX antenna patterns over the ambiguous PRFs
    TX_Amb_Arr, RX_Amb_Arr, SR_Amb_Arr, IA_Amb_Arr \
        = RASR.CalcAmbiguousAntennaPatterns_alt(
            rxAnt, 
            txAnt, 
            NumRxScanAngles * NumTxScanAngles,
            SR_Arr, 
            NN, NF, 
            Altitude, 
            AntennaOffset, 
            Wavelength, 
            prf)

#     print(SR_Amb_Arr)

    # ----------------------------------------------------------
    # for each slant range and each ambiguity count, calculate the ambiguous value
    Rasr_Amb_Scan_Arr = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_Arr = []
        for j in range(SR_Amb_Arr[i].shape[0]):
            Rasr_Amb_Arr.append(
                RASR.RasrAmb(
                    TX_Amb_Arr[i][j], 
                    RX_Amb_Arr[i][j], 
                    SR_Amb_Arr[i][j], 
                    IA_Amb_Arr[i][j]) )
        Rasr_Amb_Scan_Arr.append(Rasr_Amb_Arr)
    Rasr_Amb_Scan_Arr = np.asarray(Rasr_Amb_Scan_Arr)
#     print(Rasr_Amb_Scan_Arr)

    # ----------------------------------------------------------
    # add all the ambiguities for a given slant range together,
    # this is the total ambiguity singal for a given slant range
    Rasr_Amb_sum = []

    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_sum.append(np.nansum(Rasr_Amb_Scan_Arr[i], axis=0))
            
    Rasr_Amb_sum = np.asarray(Rasr_Amb_sum)
#     print(Rasr_Amb_sum)

    # ----------------------------------------------------------
    # calculate the Total RASR value for each slant range
    RASR_Total_Arr = [] = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        RASR_Arr = []
        for j in range(SR_Arr[i].shape[0]):
            RASR_Arr.append(RasrMain[i][j] * Rasr_Amb_sum[i][j])
        RASR_Total_Arr.append(RASR_Arr)

    RASR_Total_Arr = np.asarray(RASR_Total_Arr)
    RASR_Arr_dB = 10*np.log10(RASR_Total_Arr)
#     print(RASR_Arr_dB)

    # ----------------------------------------------------------
    # Plot RASR values vs ground
    fig, ax1 = plt.subplots(figsize = (8, 4))

    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45., 50.])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--", label="RASR")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--")

    ax1.axhline(ReqNesz, color="blue", label="Required RASR")
    
    ax1.axvline(x=Utils.rad2deg(lowIa), color="blue", linestyle="dashed")
    ax1.axvline(x=Utils.rad2deg(highIa), color="blue", linestyle="dashed")

    # ax1.set_title("RASR vs Slant Range")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"RASR (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()
    plt.tight_layout()
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_09_aesa_verification_01_rasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_09_aesa_verification_01_rasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_09_aesa_verification_01_rasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()